## Goals
 - Take the merged predictions and evaluate the prediction accuracy using the 2 different approaches
 1. Look at the anaphora tags and then cross-reference co-reference labels
 2. Use the co-reference chains directly

In [1]:
import dill
import pandas as pd

from Settings import Settings
from collections import defaultdict
from BrattEssay import ANAPHORA

from results_common import get_essays, validate_essays, tally_essay_attributes
from process_essays_coref import get_coref_processed_essays
from metrics import get_metrics_raw

# progress bar widget
from ipywidgets import IntProgress
from IPython.display import display

DATASET = "CoralBleaching" # CoralBleaching | SkinCancer

settings = Settings()
root_folder = settings.data_directory + DATASET + "/Thesis_Dataset/"
stanford_coref_predictions_folder = root_folder + "CoReference/"
print("CoRef Data: ", stanford_coref_predictions_folder)

Results Dir: /Users/simon.hughes/Google Drive/Phd/Results/
Data Dir:    /Users/simon.hughes/Google Drive/Phd/Data/
Root Dir:    /Users/simon.hughes/GitHub/NlpResearch/
Public Data: /Users/simon.hughes/GitHub/NlpResearch/Data/PublicDatasets/
CoRef Data:  /Users/simon.hughes/Google Drive/Phd/Data/CoralBleaching/Thesis_Dataset/CoReference/


In [2]:
training_essays = get_essays(stanford_coref_predictions_folder, "Training")

Found file /Users/simon.hughes/Google Drive/Phd/Data/CoralBleaching/Thesis_Dataset/CoReference/training_processed.dill


In [3]:
test_essays = get_essays(stanford_coref_predictions_folder, "Test")

Found file /Users/simon.hughes/Google Drive/Phd/Data/CoralBleaching/Thesis_Dataset/CoReference/test_processed.dill


In [4]:
all_essays = training_essays + test_essays

### Validate the Lengths

In [5]:
validate_essays(training_essays)
validate_essays(test_essays)

Essays validated
Essays validated


In [6]:
# ner_tally = tally_essay_attributes(all_essays, attribute_name="pred_ner_tags_sentences")
pos_tally = tally_essay_attributes(all_essays, attribute_name="pred_pos_tags_sentences")

## Look at the Anaphor Tags

In [7]:
from results_procesor import is_a_regular_code

cc_tally = defaultdict(int)
cr_tally = defaultdict(int)
reg_tally = defaultdict(int)
for e in all_essays:
    for sent in e.sentences:
        for wd, tags in sent:
            for t in tags:
                if is_a_regular_code(t):
                    reg_tally[t] += 1
                if ANAPHORA in t and "other" not in t:
                    if "->" in t:
                        cr_tally[t] += 1
                    elif "Anaphor:[" in t:
                        cc_tally[t] += 1

reg_tags = sorted(reg_tally.keys())
all_ana_tags = sorted(cc_tally.keys())
assert len(reg_tags) == len(all_ana_tags)
all_ana_tags

['Anaphor:[11]',
 'Anaphor:[12]',
 'Anaphor:[13]',
 'Anaphor:[14]',
 'Anaphor:[1]',
 'Anaphor:[2]',
 'Anaphor:[3]',
 'Anaphor:[4]',
 'Anaphor:[50]',
 'Anaphor:[5]',
 'Anaphor:[5b]',
 'Anaphor:[6]',
 'Anaphor:[7]']

In [30]:
LOOK_BACK = "Look back"
MAX_ANA_PHRASE = "Max ana phrase"
MAX_CHAIN_PHRASE = "Max chain phrase"
POS_ANA_FLTR = "POS ana filter"
POS_CHAIN_FLTR = "Pos chain filter"

def blank_if_none(val):
    return "-" if (val is None or not val or str(val).lower() == "none") else val

def process_sort_results(df_results):
    df_disp = df_results[["f1_score","precision","recall", MAX_ANA_PHRASE, MAX_CHAIN_PHRASE, POS_ANA_FLTR, POS_CHAIN_FLTR]]
    return df_disp.sort_values("f1_score", ascending=False)

def filter_test_results_to_best_training_results(df_train_raw, df_test_raw):
    # make sure sorted to the top result
    df_train_raw_sorted = df_train_raw.sort_values("f1_score", ascending=False, inplace=False)
    top_row = df_train_raw_sorted.iloc[0]
    filtered_df = df_test_raw[df_test_raw[LOOK_BACK]        == top_row[LOOK_BACK]]
    filtered_df = filtered_df[filtered_df[MAX_ANA_PHRASE]   == top_row[MAX_ANA_PHRASE]]
    filtered_df = filtered_df[filtered_df[MAX_CHAIN_PHRASE] == top_row[MAX_CHAIN_PHRASE]]
    filtered_df = filtered_df[filtered_df[POS_ANA_FLTR]     == top_row[POS_ANA_FLTR]]
    filtered_df = filtered_df[filtered_df[POS_CHAIN_FLTR]   == top_row[POS_CHAIN_FLTR]]
    print(len(filtered_df))
    return process_sort_results(filtered_df)

## Prepare POS Tag Filters

In [8]:
pos_nouns = set([pos for pos in pos_tally.keys() if pos.strip()[:2] == "NN"])
pos_verbs = set([pos for pos in pos_tally.keys() if pos.strip()[:2] == "VB"])
pos_pronouns = {"PRP","PRP$", "WP", "WP$"}
pos_determiners = {"DT","WDT","PDT"} # the, a, which, that, etc
pos_pron_dt = pos_pronouns | pos_determiners
# for meaning of pen treebank tags - https://www.ling.upenn.edu/courses/Fall_2003/ling001/penn_treebank_pos.html
pos_nouns, pos_verbs, pos_pronouns, pos_determiners, pos_pron_dt

({'NN', 'NNP', 'NNPS', 'NNS'},
 {'VB', 'VBD', 'VBG', 'VBN', 'VBP', 'VBZ'},
 {'PRP', 'PRP$', 'WP', 'WP$'},
 {'DT', 'PDT', 'WDT'},
 {'DT', 'PDT', 'PRP', 'PRP$', 'WDT', 'WP', 'WP$'})

In [9]:
dict_pos_filter = {
            "None": None,
            "PRN": pos_pronouns,
            "DT": pos_determiners,
            "PRN+DT": pos_pron_dt
}

dict_pos_ch_filter = {
    "None": None,
    "NN": pos_nouns,
    "VB": pos_verbs
}

In [39]:
phrase_len = [None,1,2,3,5,10,20]

# Grid Search With Anaphora Prediction Filters

### Training

In [40]:
def grid_search(essays, format_ana_tags, filter_to_predicted_tags, expected_tags):

    # set up progress bar
    max_count = len(look_back_vals)  * len(phrase_len) * len(dict_pos_filter) * len(dict_pos_ch_filter)
    iprogress_bar = IntProgress(min=0, max=max_count) # instantiate the bar
    display(iprogress_bar) # display the bar

    rows_ana = []
    
    for pos_ana_key, pos_ana_filter in dict_pos_filter.items():
        for pos_ch_key, pos_ch_filter in dict_pos_ch_filter.items():                
            for max_ana_phrase_len in phrase_len:
                for max_cref_phrase_len in phrase_len:
                    
                    proc_essays = get_coref_processed_essays(
                        essays=essays, format_ana_tags=format_ana_tags, 
                        ner_ch_filter=None, look_back_only=True,
                        filter_to_predicted_tags=filter_to_predicted_tags, 
                        max_ana_phrase_len=max_ana_phrase_len, max_cref_phrase_len=max_cref_phrase_len, 
                        pos_ana_filter=pos_ana_filter, pos_ch_filter=pos_ch_filter)
                    metrics = get_metrics_raw(proc_essays, expected_tags=expected_tags,  micro_only=True)
                    row = metrics["MICRO_F1"]
                    row[MAX_ANA_PHRASE] = blank_if_none(max_ana_phrase_len)
                    row[MAX_CHAIN_PHRASE] = blank_if_none(max_cref_phrase_len)
                    row[POS_ANA_FLTR]     = blank_if_none(pos_ana_key)
                    row[POS_CHAIN_FLTR]   = blank_if_none(pos_ch_key)
                    rows_ana.append(row)
                    iprogress_bar.value += 1

    df_results = pd.DataFrame(rows_ana)
    return df_results

In [ ]:
filter_to_predicted_tags = True
format_ana_tags=True # Format tags with Anaphora[xyz]

df_train_raw = grid_search(essays=training_essays, 
                           expected_tags=all_ana_tags,
                           filter_to_predicted_tags=filter_to_predicted_tags, format_ana_tags=format_ana_tags)

IntProgress(value=0, max=168)

KeyboardInterrupt: 

In [ ]:
process_sort_results(df_train_raw).head()

### Test

In [ ]:
filter_to_predicted_tags = True
format_ana_tags=True # Format tags with Anaphora[xyz]

df_test_raw = grid_search(essays=test_essays, 
                          expected_tags=all_ana_tags,
                           filter_to_predicted_tags=filter_to_predicted_tags, format_ana_tags=format_ana_tags)

In [ ]:
process_sort_results(df_test_raw).head()

In [ ]:
filter_test_results_to_best_training_results(df_train_raw, df_test_raw)

## Grid Search without Anaphora Predictions

### Training

In [ ]:
filter_to_predicted_tags = False
format_ana_tags=True # Format tags with Anaphora[xyz]

df_train_raw_cref = grid_search(essays=training_essays, 
                                expected_tags=all_ana_tags,
                           filter_to_predicted_tags=filter_to_predicted_tags, format_ana_tags=format_ana_tags)

In [ ]:
process_sort_results(df_train_raw_cref).head()

### Test

In [ ]:
filter_to_predicted_tags = False
format_ana_tags=True # Format tags with Anaphora[xyz]

df_test_raw_cref = grid_search(essays=test_essays, 
                               expected_tags=all_ana_tags,
                           filter_to_predicted_tags=filter_to_predicted_tags, format_ana_tags=format_ana_tags)

In [ ]:
process_sort_results(df_test_raw_cref).head()

In [ ]:
filter_test_results_to_best_training_results(df_train_raw_cref, df_test_raw_cref)

## TODO
- ~~Make look-back-only always true~~
  - **instead, add an option to pick the nearest preceding referent only**
- ~~add a length filter to look at the length of the anaphor as well as the antecedents~~

In [86]:
from process_essays_coref import build_segmented_chain

corefid_chain = build_segmented_chain(training_essays[10])
corefid_chain

{'1': [[(4, 6), (4, 7), (4, 8)],
  [(5, 0), (5, 1)],
  [(5, 23), (5, 24), (5, 25)]],
 '2': [[(1, 11), (1, 12), (1, 13)]],
 '3': [[(3, 0), (3, 1)], [(3, 9)]]}

In [91]:
chain = corefid_chain["1"]
for phrase in chain:
    print(phrase)
    print(phrase[0], phrase[-1])
    print()

[(4, 6), (4, 7), (4, 8)]
(4, 6) (4, 8)

[(5, 0), (5, 1)]
(5, 0) (5, 1)

[(5, 23), (5, 24), (5, 25)]
(5, 23) (5, 25)



In [94]:
def find_coref_length(corefid_chain,  corefid, sent_ix, word_ix):
    chain = corefid_chain[corefid]
    length = 0
    for phrase in chain:
        first_pair = phrase[0]
        last_pair = phrase[-1]
        first_sent, first_word = first_pair
        last_sent,  last_word = last_pair
        
        # within same sentence
        if sent_ix >= first_sent and sent_ix <= last_sent:
            assert first_sent == last_sent, "Phrase spans different sentences"
            if word_ix >= first_word and word_ix <= last_word:
                length = max(length, (last_word-first_word)+1)
    assert length > 0, "can't find matching coref"
    return length

In [100]:
find_coref_length(corefid_chain, "1", 5, 26)

0